In [26]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split

import h2o
from h2o.automl import H2OAutoML
#from tqdm import tqdm

In [27]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_391"; Java(TM) SE Runtime Environment (build 1.8.0_391-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.391-b13, mixed mode)
  Starting server from /Users/emma/miniconda3/envs/h20/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/ns/rjvmq7zn3cggtjmkkj8bhfsc0000gn/T/tmp_1omekrv
  JVM stdout: /var/folders/ns/rjvmq7zn3cggtjmkkj8bhfsc0000gn/T/tmp_1omekrv/h2o_emma_started_from_python.out
  JVM stderr: /var/folders/ns/rjvmq7zn3cggtjmkkj8bhfsc0000gn/T/tmp_1omekrv/h2o_emma_started_from_python.err
  Server is running at http://127.0.0.1:54325
Connecting to H2O server at http://127.0.0.1:54325 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,7 months and 3 days
H2O_cluster_name:,H2O_from_python_emma_i5pd5x
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.770 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [28]:
df = pd.read_csv("../data/translation/classified2.csv", sep=";")
#dfo = pd.read_csv("../data/translation/completo.csv")
df

,Unnamed: 0,date,time,name,message,emotion
0,0,2023-04-09,13:33:57,Em,Quan acabo et truco,12
1,1,2019-12-04,17:32:34,Em,Tia pero escucha haz abdominales si queires ha...,14
2,2,2020-06-16,17:12:10,Em,Si ara mateix,9
3,3,2018-08-07,15:10:52,Em,41577827Z,13
4,4,2019-03-10,10:57:46,Em,Ya me contaras como ha ido,10
...,...,...,...,...,...,...
993,995,2021-11-25,21:14:54,Em,Quina vidaaa!!! Estic same literal,8
994,996,2023-08-03,23:22:13,Em,Como ha ido tu diaa?,15
995,997,2020-07-02,13:38:15,Em,Okay,13
996,998,2021-02-02,20:54:18,Em,Etc,13


In [29]:
train, test = train_test_split(df, test_size=0.25, random_state=42)

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [30]:
X = ["message"]
y = "emotion"

In [34]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [35]:
# Run AutoML for 20 base models
automl = H2OAutoML(max_models=4, seed=1)
automl.train(x=X, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = automl.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)
lb

AutoML progress: |
15:53:31.84: _train param, Dropping bad and constant columns: [message]
15:53:31.85: _train param, Training data must have at least 2 features (incl. response).
15:53:31.87: _train param, Dropping bad and constant columns: [message]
15:53:31.87: _train param, Training data must have at least 2 features (incl. response).
15:53:31.92: _train param, Dropping bad and constant columns: [message]
15:53:31.92: _train param, Training data must have at least 2 features (incl. response).
15:53:31.94: _train param, Dropping bad and constant columns: [message]
15:53:31.95: _train param, Training data must have at least 2 features (incl. response).
15:53:31.97: _train param, Dropping bad and constant columns: [message]
15:53:31.97: _train param, Training data must have at least 2 features (incl. response).
15:53:31.99: _train param, Dropping bad and constant columns: [message]
15:53:31.99: _train param, Training data must have at least 2 features (incl. response).
15:53:31.101: _

OSError: Job with key $03017f00000136d4ffffffff$_9a0db181b2dbab034d5185b42c35418c failed with an exception: water.exceptions.H2OAutoMLException: Aborting AutoML after too many consecutive model failures
stacktrace: 
water.exceptions.H2OAutoMLException: Aborting AutoML after too many consecutive model failures
	at ai.h2o.automl.AutoML.learn(AutoML.java:776)
	at ai.h2o.automl.AutoML.run(AutoML.java:494)
	at ai.h2o.automl.H2OJob$1.compute2(H2OJob.java:33)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1677)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:976)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1479)
	at jsr166y.ForkJoinWorkerThread.run(ForkJoinWorkerThread.java:104)


In [36]:
m = h2o.get_model('GBM_1_AutoML_1_20231201_150850')
m

H2OResponseError: Server error water.exceptions.H2OKeyNotFoundArgumentException:
  Error: Object 'GBM_1_AutoML_1_20231201_150850' not found for argument: key
  Request: GET /3/Models/GBM_1_AutoML_1_20231201_150850


In [ ]:
model_path = h2o.save_model(model=m, path="ola", force=True)
saved_model = h2o.load_model("ola/GBM_1_AutoML_1_20231201_140102")

NameError: name 'm' is not defined

In [19]:
df.drop(columns="emotion").iloc[0]

Unnamed: 0                      0
date                   2023-04-09
time                     13:33:57
name                           Em
message       Quan acabo et truco
Name: 0, dtype: object

In [20]:
# 1. Split
dfo = pd.read_csv("../data/translation/completo.csv")

train, test = train_test_split(df, test_size=0.15, random_state=42)

# 2. X & y
X = dfo[["message"]]
y = None

# 3. X, y: H20 objects
to_predict = h2o.H2OFrame(X)

# 4. Predict
y_pred = saved_model.predict(to_predict)["predict"]

y_pred_as_pandas = y_pred.as_data_frame()

dfo["emotions"] = y_pred_as_pandas
"""
# 5. Results back to pandas
to_predict_as_pandas = to_predict.as_data_frame()
y_pred_as_pandas = y_pred.as_data_frame()

print(list(to_predict_as_pandas))
print(list(y_pred_as_pandas))

# 6. Build df
new_df = pd.DataFrame({"message": list(to_predict_as_pandas), "predicted_emotion": list(y_pred_as_pandas)})
new_df
"""

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


NameError: name 'saved_model' is not defined

In [ ]:
emotions_2 = {1: 'sadness',
                2: 'joy',
                3: 'fear',
                4: 'anger',
                5: 'surprise',
                6: 'defensive',
                7: 'nostalgia',
                8: 'hapiness',
                9: 'stressed',
                10: 'expectant',
                11: 'insecure',
                12: 'assertivity',
                13: 'neutral',
                14: 'excitement',
                15: 'comprehensive',
                16: 'helpless',
                17: 'love',
                18: 'deception',
                19: 'doubtful',
                20: 'picarona'}

In [ ]:
dfo.emotions.value_counts()
dfo["emotions_names"] = dfo["emotions"].map(emotions_2)
dfo.to_csv("../data/final df/h2o2.csv")

In [ ]:
dfo.emotions.value_counts()

emotions
12    111803
13      3260
8       1863
2        773
17       246
6        180
15       129
9        113
19       106
4         65
14        49
10        33
1         26
18        24
11        20
20        17
5         14
7         11
3         11
16         9
Name: count, dtype: int64